# 2D flying-snake at Re=1000 and AoA=40deg (snappyHexMesh + OpenFOAM-2.3.0)

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.openfoam.simulation import OpenFOAMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1


---

### Description

Flying-snake at Re=1000 and angle-of-attack 40deg.

The mesh was generated with the OpenFOAM mesh utility, snappyHexMesh and contains about 3.4-million cells.

---

### General information

* software: `OpenFOAM-2.3.0`
* post-processing software: `ParaView-4.1` (included in `OpenFOAM-2.3.1`)
* submission date: 2015/11/10 (mesnardo)
* running date: 2015/11/10
* machine: `Colonial One` (short queue, 32 processes, 2 nodes)

In [ ]:
simulation = OpenFOAMSimulation(description='IcoFOAM')

---

### Computational domain and mesh

* `snappyHexMesh`, 3.4M cells (mostly hexahedra)
* domain: `[-15, 15]x[-15, 15]` (cell-width 30/120=0.25)
* refinement box 1: `[-1, 10]x[-2, 2]` (cell-width 0.25/2^6=0.004)
* refinement box 2: `[-2, 15]x[-4, 4]` (cell-width 0.25/2^4=0.016)

The mesh was generated using `features` for refinement boxes (`.obj` files containing the box coordinates located in the folder `constant/triSurface`).

Because `snappyHexMesh` is a 3D meshing tool, we only try to refine as close as possible from the `front` patch. Then, we only keep `front` and extrude in the z-direction, to get a 3D mesh with only one cell in the z-direction.

---

### Boundary conditions

In [ ]:
boundary_info = '0/U'
%cat $boundary_info
boundary_info = '0/p'
%cat $boundary_info

---

### Simulation parameters

In [ ]:
simulation_info = 'system/controlDict'
%cat $simulation_info
simulation_info = 'system/fvSolution'
%cat $simulation_info
simulation_info = 'system/fvScheme'
%cat $simulation_info

---

### Running the case

To generate the mesh:
    > runSnappyHexMesh.sh
    
To run the case:
    > mpirun -n <nprocs> icoFOAM -parallel > log.icoFoam

To reconstruct the parallel solution:
    > reconstructPar

To generate the vorticity variable:
    > vorticity

---

### Post-processing

We computed 100 non-dimensional time-units in about 23h.

#### Forces

In [ ]:
simulation.read_forces(display_coefficients=True)
simulation.get_mean_forces(limits=[32.0, 64.0])
simulation.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True)

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe1000AoA40',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 64.0])
krishnan.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan,
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
dataframes = dataframe.append(dataframe2)
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

In [ ]:
simulation.plot_field_contours_paraview('vorticity',
                                        field_range=(-5.0, 5.0),
                                        view=(-2.0, -5.0, 15.0, 5.0),
                                        width=600,
                                        colormap='RdBu_r')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'vorticity_-2.00_-5.00_15.00_5.00'))